In [1]:
import os
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as tt
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
from tqdm.notebook import tqdm
from torchvision.utils import save_image
from torchvision.utils import make_grid
import torchaudio
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
from time import time
import warnings
warnings.filterwarnings("ignore")

/home/kamenev_v/miniconda3/envs/gpuenv/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [2]:
os.chdir('/home/kamenev_v/Deepfake_proj/DeepFakeDetection/')

In [3]:
from lib.data.dataset import *
from lib.utils import *
from lib.training.train_model import *
from lib.models.FTFD import *
from lib.models.BaseDetector import *
from lib.models.PFD import *

In [4]:
from facenet_pytorch import MTCNN

In [5]:
image_size = 96
audio_size_up = 96
audio_size_d = 768
# transform_image = tt.Compose([tt.ToPILImage(),tt.Resize(image_size),tt.CenterCrop(image_size),
#                               tt.ToTensor(),
#                                  tt.RandomHorizontalFlip(p=0.3),
#                                      tt.RandomRotation(degrees=7),
#                                      AddGaussianNoise(p=0.5,mean=0,std=0.1)])
transform_image = tt.Compose([tt.Resize(image_size),tt.CenterCrop(image_size)])
                                 # tt.RandomHorizontalFlip(p=0.3),
                                 #     tt.RandomRotation(degrees=7),
                                 #     AddGaussianNoise(p=0.3,mean=0.5,std=0.1)])
transform_upsampled_audio = tt.Compose([tt.Resize(audio_size_up),tt.CenterCrop(audio_size_up)])
transform_default_audio = tt.Compose([tt.Resize(audio_size_d),tt.CenterCrop(audio_size_d)])

In [6]:
mod = DeepFakeDetector(num_of_input_imgs=3,
                       confidence_face=0.95,patch_mode=False)
res = mod.predict_video('../full_dataset/youtube/0001_01_001_adam_sandler.avi')
res

tensor([[0.6420]], device='cuda:0', grad_fn=<SigmoidBackward>)


'This is fake'

In [7]:
res1

NameError: name 'res1' is not defined

In [ ]:
#frames,audio,info = read_video('../full_dataset/FakeAVCeleb/RealVideo-FakeAudio/African/women/id00371/00099_fake.mp4')

In [ ]:
#frames1,audio1,info1 = read_video('../full_dataset/FakeAVCeleb/RealVideo-FakeAudio/African/men/id00076/00109_fake.mp4')

In [ ]:
#plot_waveform(audio1.squeeze(dim=0),44100)

In [ ]:
_,audio,info = read_video('../full_dataset/FakeAVCeleb/FakeVideo-FakeAudio/African/men/id00076/00109_10_id00476_wavtolip.mp4')

In [ ]:
DEVICE = torch.device('cuda')


In [8]:
TRAIN = pd.read_csv('lib/data/FINAL_TRAIN_VA.csv')
TEST = pd.read_csv('lib/data/FINAL_TEST_VA.csv')
#TRAIN['path'] = TRAIN['path'].apply(lambda x: '/'.join(x.split('/')[3:]))

In [10]:
TRAIN

,path,video,audio,class
0,Fake/id09143_1371__video1_audio0,1.0,0.0,1.0
1,Fake/id02587_761__video0_audio1,0.0,1.0,1.0
2,True/id06355_432__video0_audio0,0.0,0.0,0.0
3,Fake/id00169_2279__video1_audio1,1.0,1.0,1.0
4,Fake/id04774_726__video0_audio1,0.0,1.0,1.0
...,...,...,...,...
2055,Fake/id04374_2166__video1_audio1,1.0,1.0,1.0
2056,Fake/id05231_2202__video1_audio1,1.0,1.0,1.0
2057,Fake/id02553_715__video0_audio1,0.0,1.0,1.0
2058,Fake/id01207_521__video0_audio1,0.0,1.0,1.0


In [ ]:
TRAIN = pd.concat([TRAIN[TRAIN['class']==1].sample(467),TRAIN[TRAIN['class']==0]])
TEST = pd.concat([TEST[TEST['class']==1].sample(109),TEST[TEST['class']==0]])

In [ ]:
TRAIN['path'] = TRAIN['path'].apply(lambda x: 'lib/data/CUSTOMV2/'+x)
TEST['path'] = TEST['path'].apply(lambda x: 'lib/data/CUSTOMV2/'+x)

In [ ]:
model = MultimodalModel(3*3,audio_size_default=768,classify=False,return_patch=True) ### IMAGE (T,96,96,3), AUDIO (384,384)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(params = model.parameters())
loss = nn.BCELoss()

# pytorch_total_params = sum(p.numel() for p in model.parameters())
#pytorch_total_params

In [ ]:
# TRAIN['class'] = TRAIN['class'].apply(lambda x: patch_apply(x))
# TEST['class'] = TEST['class'].apply(lambda x: patch_apply(x))

In [ ]:
train_dataset = CelebVADataset(TRAIN,3,transform_image,transform_upsampled_audio,
                              transform_default_audio)
test_dataset = CelebVADataset(TEST,3,transform_upsampled_audio,transform_upsampled_audio,
                             transform_default_audio)

In [ ]:
# k = train(model=model,opt=optimizer,loss_fn=loss,epochs=50,
#           data_tr=train_dataset,data_val=test_dataset,batch=40,patience=7,
#           path_to_save='lib/models/FTFD_768_final.pth',patch_v=False,DEVICE=DEVICE)

In [ ]:
from lib.metrics.evaluate import Evaluator
checker = Evaluator(type_of_model='av',loss_fn=nn.BCELoss(),patch_v=False)
checker.estimate_dataset(test_dataset,model)